In [ ]:
import random

def calculate_insertion_prob(queue_num, total_queues, time, max_time):
    # Insertion probability grows from 10% to 70% and peaks based on the queue/stack number
    peak_time = (queue_num / total_queues) * max_time
    if time <= peak_time:
        return 0.1 + 0.6 * (time / peak_time)  # Linear growth to peak
    else:
        return 0.7 - 0.7 * ((time - peak_time) / (max_time - peak_time))  # Linear decrease to 0%

def calculate_deletion_prob(queue_num, total_queues, time, max_time):
    # Deletion probability peaks 10% after insertion and decreases to 10% by the end
    peak_time = (queue_num / total_queues) * max_time * 1.1  # Peak happens 10% later
    if time <= peak_time:
        return 0.1 + 0.6 * (time / peak_time)  # Linear growth to peak
    else:
        return 0.7 - 0.6 * ((time - peak_time) / (max_time - peak_time)) + 0.1  # Decrease to 10%

def generate_operation_sequence(q, s, max_time=10000):
    operations = []
    unique_id = 1  # This will be the unique ID inserted into queues/stacks

    # Iterate through time from t00001 to t10000
    for time in range(1, max_time + 1):
        timestamp = f"t{time:05d}"

        # For each queue, decide insert or delete based on probabilities
        for queue_num in range(1, q + 1):
            insertion_prob = calculate_insertion_prob(queue_num, q, time, max_time)
            deletion_prob = calculate_deletion_prob(queue_num, q, time, max_time)

            if random.random() < insertion_prob:
                operations.append(f"{timestamp} enqueue Q{queue_num}, {unique_id}")
                unique_id += 1
            elif random.random() < deletion_prob:
                operations.append(f"{timestamp} dequeue Q{queue_num}")

        # For each stack, decide insert or delete based on probabilities
        for stack_num in range(1, s + 1):
            insertion_prob = calculate_insertion_prob(stack_num, s, time, max_time)
            deletion_prob = calculate_deletion_prob(stack_num, s, time, max_time)

            if random.random() < insertion_prob:
                operations.append(f"{timestamp} push S{stack_num}, {unique_id}")
                unique_id += 1
            elif random.random() < deletion_prob:
                operations.append(f"{timestamp} pop S{stack_num}")

    return operations

def exercise_2(operations, q, s):

    queue_items = [[] for _ in range(q)]
    stack_items = [[] for _ in range(s)]


    queue_total_stay = [0] * q
    stack_total_stay = [0] * s
    queue_max_stay = [0] * q
    stack_max_stay = [0] * s
    queue_max_item = [None] * q
    stack_max_item = [None] * s


    queue_item_count = [0] * q
    stack_item_count = [0] * s

    current_time = 0

    for op in operations:
        operation = op.split()
        time = int(operation[0].split('t')[1])
        current_time = time

        if "enqueue" in op:
            queue_number = int(operation[2].split('Q')[1][0]) - 1
            item_id = int(operation[3])
            queue_items[queue_number].append({'item_id': item_id, 'insert_time': time})

        elif "dequeue" in op:
            queue_number = int(operation[2].split('Q')[1][0]) - 1
            if queue_items[queue_number]:
                item = queue_items[queue_number].pop(0)
                stay_duration = time - item['insert_time']
                queue_total_stay[queue_number] += stay_duration
                queue_item_count[queue_number] += 1
                if stay_duration > queue_max_stay[queue_number]:
                    queue_max_stay[queue_number] = stay_duration
                    queue_max_item[queue_number] = item['item_id']

        if "push" in op:
            stack_number = int(operation[2].split('S')[1][0]) - 1
            item_id = int(operation[3])
            stack_items[stack_number].append({'item_id': item_id, 'insert_time': time})

        elif "pop" in op:
            stack_number = int(operation[2].split('S')[1][0]) - 1
            if stack_items[stack_number]:
                item = stack_items[stack_number].pop()  # Pop from the top
                stay_duration = time - item['insert_time']
                stack_total_stay[stack_number] += stay_duration
                stack_item_count[stack_number] += 1
                if stay_duration > stack_max_stay[stack_number]:
                    stack_max_stay[stack_number] = stay_duration
                    stack_max_item[stack_number] = item['item_id']

    # After finishing operations, empty any remaining items
    while any(queue_items) or any(stack_items):
        current_time += 1  # Increment time for each emptying step

        for queue_number in range(q):
            if queue_items[queue_number]:
                item = queue_items[queue_number].pop(0)  # Dequeue remaining items
                stay_duration = current_time - item['insert_time']
                queue_total_stay[queue_number] += stay_duration
                queue_item_count[queue_number] += 1
                if stay_duration > queue_max_stay[queue_number]:
                    queue_max_stay[queue_number] = stay_duration
                    queue_max_item[queue_number] = item['item_id']

        for stack_number in range(s):
            if stack_items[stack_number]:
                item = stack_items[stack_number].pop()  # Pop remaining items
                stay_duration = current_time - item['insert_time']
                stack_total_stay[stack_number] += stay_duration
                stack_item_count[stack_number] += 1
                if stay_duration > stack_max_stay[stack_number]:
                    stack_max_stay[stack_number] = stay_duration
                    stack_max_item[stack_number] = item['item_id']


    print("Queue Results:")
    for i in range(q):
        if queue_item_count[i] > 0:
            avg_stay = queue_total_stay[i] / queue_item_count[i]
            print(f"Queue Q{i + 1}: Avg time stayed = {avg_stay}, Longest stay = {queue_max_stay[i]} (Item {queue_max_item[i]})")
        else:
            print(f"Queue Q{i + 1}: No items processed.")

    print("\nStack Results:")
    for i in range(s):
        if stack_item_count[i] > 0:
            avg_stay = stack_total_stay[i] / stack_item_count[i]
            print(f"Stack S{i + 1}: Avg time stayed = {avg_stay}, Longest stay = {stack_max_stay[i]} (Item {stack_max_item[i]})")
        else:
            print(f"Stack S{i + 1}: No items processed.")

    return

if __name__ == "__main__":
    q_1 = 5 #5 queues
    s_1 = 3 #3 stacks

    q_2 = 3 #3 queues
    s_2 = 5 #5 stacks

    operations = generate_operation_sequence(q_1, s_1)
    print("Exercise 2 a: \n")
    exercise_2(operations,q_1,s_1)
    print("Exercise 2 b: \n")
    operations_1 = generate_operation_sequence(q_2, s_2)
    exercise_2(operations_1,q_2,s_2)